In [21]:
import requests
import re
import json
import pandas as pd
from datetime import datetime

In [22]:
global list_rottentomatoes_reviews
list_rottentomatoes_reviews = []

In [23]:
def getReviews(endCursor, movie_Id):
    r = requests.get(f"https://www.rottentomatoes.com/napi/movie/{movie_Id}/reviews/user",
    params = {
        "direction": "next",
        "endCursor": endCursor,
        "startCursor": ""
    })
    return r.json()

In [24]:
def extract_reviews(label_name, movie_name):
    try:
        #r = requests.get("https://www.rottentomatoes.com/m/"+movie_name+"/reviews?type=user")
        r = requests.get("https://www.rottentomatoes.com/m/"+movie_name+"/reviews?type=top_critics")
        movieId = re.findall('\"movieId\":\"[0-9a-z-]*\"', r.text)
        print(movie_name, ":", movieId)
        movieId = movieId[0].split(':')[1].replace('"','')
        reviews = []
        result = {}
        for i in range(0, 5):
            result = getReviews(result["pageInfo"]["endCursor"] if i != 0  else "", movieId)
            reviews.extend([t for t in result["reviews"]])
        print(f"got {len(reviews)} reviews")
        df_reviews = pd.DataFrame(reviews)
        df_reviews["label_name"] = label_name
        df_reviews["movie_name"] = movie_name
        list_rottentomatoes_reviews.append(df_reviews[["label_name", "movie_name", "review", "rating", "timeFromCreation"]])
    except Exception as e:
        print("Exception Caught: ", e)

In [25]:
df_movies_id = pd.read_csv("input/imdb_movie_list.csv")
df_movies_id

,Universe,Movie_Name,IMDB_Movie_Id,Movie_Name_Key
0,Star Wars,A New Hope,76759,star_wars_a_new_hope
1,Star Wars,The Empire Strikes Back,80684,star_wars_the_empire_strikes_back
2,Star Wars,Return of the Jedi,86190,star_wars_episode_vi_return_of_the_jedi
3,Star Wars,The Phantom Menace,120915,star_wars_episode_i_the_phantom_menace
4,Star Wars,Attack of the Clones,121765,star_wars_episode_ii_attack_of_the_clones
...,...,...,...,...
59,Wizarding World,Harry Potter and the Half-Blood Prince,417741,harry_potter_and_the_half_blood_prince
60,Wizarding World,Harry Potter and the Deathly Hallows – Part 1,926084,harry_potter_and_the_deathly_hallows_part_1
61,Wizarding World,Harry Potter and the Deathly Hallows – Part 2,1201607,harry_potter_and_the_deathly_hallows_part_2
62,Wizarding World,Fantastic Beasts and Where to Find Them,3183660,fantastic_beasts_and_where_to_find_them


In [26]:
s = str(datetime.now())
s = s[:16].replace(" ", "_").replace(":","")

In [27]:
for i in df_movies_id.index:
    extract_reviews(df_movies_id["Universe"][i].strip(), df_movies_id["Movie_Name_Key"][i].strip())

star_wars_a_new_hope : ['"movieId":"e2bff218-467b-3595-ae11-c62d690641ad"']
got 50 reviews
star_wars_the_empire_strikes_back : ['"movieId":"4ecba525-0751-3a00-8b8e-c7a10ff0e385"']
got 50 reviews
star_wars_episode_vi_return_of_the_jedi : ['"movieId":"70e5cd12-1b65-37df-b95b-7a5728faf7c1"']
got 50 reviews
star_wars_episode_i_the_phantom_menace : ['"movieId":"27aa565e-c8df-3268-9df1-571deb443681"']
got 50 reviews
star_wars_episode_ii_attack_of_the_clones : ['"movieId":"4490561c-8395-30ec-8b24-dae566a80c1c"']
got 50 reviews
star_wars_episode_iii_revenge_of_the_sith : ['"movieId":"2ce28849-980d-3c3b-bcca-bd0b38dfec9f"']
got 50 reviews
star_wars_episode_vii_the_force_awakens : ['"movieId":"83b1f72e-6dd5-3538-89dd-b10e57016fbf"']
got 50 reviews
star_wars_the_last_jedi : ['"movieId":"ad34316e-4d9c-3f65-a3cf-4a2174e294a1"']
got 50 reviews
star_wars_the_rise_of_skywalker : ['"movieId":"d7083795-3ab7-3b17-9717-bbe6401ffd79"']
got 50 reviews
clone_wars : ['"movieId":"5275ca20-803b-3f17-a40d-04e7df

In [30]:
df_rottentomatoes_reviews = pd.concat(list_rottentomatoes_reviews)
df_rottentomatoes_reviews.to_csv("df_rottentomatoes_reviews_"+s+".csv", index = False)

In [31]:
df_rottentomatoes_reviews

,label_name,movie_name,review,rating,timeFromCreation
0,Star Wars,star_wars_a_new_hope,"first and absoulte classic star wars movie, ah...",5.0,5d ago
1,Star Wars,star_wars_a_new_hope,George Lucas does the impossible by creating a...,5.0,"Mar 29, 2022"
2,Star Wars,star_wars_a_new_hope,The film that started it all. A new hope has g...,5.0,"Mar 22, 2022"
3,Star Wars,star_wars_a_new_hope,Can’t do justice with a review in 2022 that ha...,5.0,"Mar 16, 2022"
4,Star Wars,star_wars_a_new_hope,A milestone in the cinematic universe. Great s...,5.0,"Mar 13, 2022"
...,...,...,...,...,...
45,Wizarding World,fantastic_beasts_the_crimes_of_grindelwald,"The movie's not bad, there is cool action scen...",3.0,"Aug 05, 2021"
46,Wizarding World,fantastic_beasts_the_crimes_of_grindelwald,I seem to be one of the rare people that genui...,4.0,"Jul 24, 2021"
47,Wizarding World,fantastic_beasts_the_crimes_of_grindelwald,The crimes of Grindelwald is a very good film ...,3.0,"Jul 19, 2021"
48,Wizarding World,fantastic_beasts_the_crimes_of_grindelwald,Little to no redeeming factors in this boring ...,1.5,"Jul 13, 2021"


In [45]:
df_rottentomatoes_reviews['length'] = df_rottentomatoes_reviews['review'].str.len()

In [64]:
dfg = df_rottentomatoes_reviews.groupby(["label_name","movie_name"])[['review','length']]
list_max_reviews = []
for g in dfg:
    review_length = min(list(g[1]['review'].str.len().nlargest(10)))
    d = g[1]
    list_max_reviews.append(d[d['length'] >= review_length])

In [65]:
df_rottentomatoes_reviews_max = pd.concat(list_max_reviews)

In [66]:
df_rottentomatoes_reviews_max

,label_name,movie_name,review,rating,timeFromCreation,length
4,DC Extended Universe,aquaman_2018,"It was ok, but the film wanted to be more than...",3.0,"Mar 11, 2022",348
13,DC Extended Universe,aquaman_2018,Completely inoffensive and acceptable but noth...,3.0,"Feb 26, 2022",332
15,DC Extended Universe,aquaman_2018,The son of a humble lighthouse keeper and an e...,2.5,"Feb 12, 2022",2024
19,DC Extended Universe,aquaman_2018,I felt the plot could've been better developed...,4.0,"Feb 04, 2022",567
26,DC Extended Universe,aquaman_2018,It's about time a less-popular member of the J...,4.0,"Jan 17, 2022",810
...,...,...,...,...,...,...
30,Wizarding World,harry_potter_and_the_sorcerers_stone,"Sin duda esta película, llegó para revoluciona...",3.0,"Nov 24, 2021",928
38,Wizarding World,harry_potter_and_the_sorcerers_stone,"I liked this movie, but it had some serious fl...",2.0,"Oct 30, 2021",849
46,Wizarding World,harry_potter_and_the_sorcerers_stone,You probably couldn't have hoped for a better ...,4.0,"Aug 17, 2021",799
47,Wizarding World,harry_potter_and_the_sorcerers_stone,"""Hary Potter"" is the beginning of everything, ...",4.0,"Aug 17, 2021",1197


In [68]:
df_rottentomatoes_reviews_max.to_csv("df_rottentomatoes_reviews_max_"+s+".csv", index = False)